# Imports

In [ ]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import pandas as pd
from datetime import datetime as DT
import xarray as xr

In [ ]:
from imod import msw
from imod import mf6
import primod

In [ ]:
import flopy as fp

In [ ]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)
IL.reload(primod)
IL.reload(msw)
IL.reload(mf6)

In [ ]:
# Additional imports for shapefile creation
from shapely.geometry import LineString, Point
import geopandas as gpd
import re

# Options + Basics

In [ ]:
MdlN = 'NBr30'

In [ ]:
# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax = [float(i) for i in d_INI['WINDOW'].split(',')]
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])

In [ ]:
d_Pa

# Read SFR

## Read file

In [ ]:
Pa_SFR = d_Pa['SFR']

In [ ]:
l_Lns = U.r_Txt_Lns(Pa_SFR)

## PACKAGEDATA 

### Read into DF

In [ ]:
PkgDt_start = next(i for i,l in enumerate(l_Lns) if 'BEGIN PACKAGEDATA' in l)+2
PkgDt_end = next(i for i,l in enumerate(l_Lns) if 'END PACKAGEDATA' in l)
PkgDt_Cols = ['ifno', 'L', 'R', 'C', 'rlen', 'rwid', 'rgrd', 'rtp', 'rbth', 'rhk', 'man', 'ncon', 'ustrf', 'ndv', 'aux', 'X', 'Y']
PkgDt_data = [l.split() for l in l_Lns[PkgDt_start:PkgDt_end] if l.strip() and not l.strip().startswith('#')]

for row in PkgDt_data: # Robust fix: if only one 'NONE' and it's at index 1, replace with three 'NONE's
    if row.count('NONE') == 1 and row[1] == 'NONE':
        row[1:2] = ['NONE','NONE','NONE']

DF_PkgDt = pd.DataFrame(PkgDt_data, columns=PkgDt_Cols)
DF_PkgDt

### Cleanup

In [ ]:
DF_PkgDt = DF_PkgDt.replace(["NONE", "", "NaN", "nan"], pd.NA) # 1) normalize NA-like tokens and strip spaces
DF_PkgDt = DF_PkgDt.apply(lambda s: s.str.strip() if s.dtype == "object" else s)

l_Num_Cols = [c for c in DF_PkgDt.columns if c != 'aux']  # 2) choose numeric columns and coerce
DF_PkgDt[l_Num_Cols] = DF_PkgDt[l_Num_Cols].apply(pd.to_numeric)

# 3) optional: get nullable ints/floats
DF_PkgDt = DF_PkgDt.convert_dtypes()
# or enforce ints: DF_PkgDt[num_cols] = DF_PkgDt[num_cols].astype("Int64")


In [ ]:
DF_PkgDt.dtypes

## CONNECTIONDATA

In [ ]:
Conn_start = next(i for i,l in enumerate(l_Lns) if 'BEGIN CONNECTIONDATA' in l)+1
Conn_end = next(i for i,l in enumerate(l_Lns) if 'END CONNECTIONDATA' in l)
Conn_data = [(int(parts[0]), [int(x) for x in parts[1:]])
             for l in l_Lns[Conn_start+1:Conn_end]
             if (parts := l.strip().split())]

DF_Conn = pd.DataFrame(Conn_data, columns=['reach','connections'])
DF_Conn['downstream'] = DF_Conn['connections'].apply(lambda l_Conns: next((-x for x in l_Conns if x < 0), None))
DF_Conn['downstream'] = DF_Conn['downstream'].astype("Int64")
DF_Conn

## Merge

In [ ]:
DF = pd.merge(DF_PkgDt, DF_Conn[['reach', 'downstream']], left_on='ifno', right_on='reach', how='left')

In [ ]:
def make_geom(row, DF):
    if pd.notna(row['downstream']):
        # Get downstream X,Y
        DS = DF.loc[DF['ifno'] == row['downstream']]
        if not DS.empty:
            return LineString([(row['X'], row['Y']), (DS.iloc[0]['X'], DS.iloc[0]['Y'])])
    # If no downstream, return Point (or buffer for circle)
    return Point(row['X'], row['Y']).buffer(dx)

GDF = DF.copy()
GDF['geometry'] = GDF.apply(lambda r: make_geom(r, GDF), axis=1)
GDF = gpd.GeoDataFrame(GDF, geometry='geometry', crs=28992)  # Set CRS as needed
GDF.head()

In [ ]:
GDF.head()

In [ ]:
GDF.insert(0, 'reach')

In [ ]:
os.getcwd()

In [ ]:
GDF.to_file('SFR_lines.shp')


In [ ]:
DF